In [92]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [93]:
class Vertice:
    
    def __init__(self, valor, direcionado=True):
        self.__valor = valor
        self.__direcionado = direcionado
        self.__arestas = set()
    
    def getValor(self):
        return self.__valor
    
    def setValor(self, valor):
        self.__valor = valor
        
    def getArestas(self):
        return self.__arestas
    
    def adicionarAresta(self, aresta):
        self.__arestas.add(aresta)
        
    def getArestasSaida(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasDeSaida = []
        for aresta in self.__arestas:
            if aresta.getvOrigem() == self:
                arestasDeSaida.append(aresta)
        return arestasDeSaida
    
    def getArestasEntrada(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasSaida = []
        for aresta in self.__arestas:
            if aresta.getvDestino() == self:
                arestasSaida.append(aresta)
        return arestasSaida
    
    def getGrau(self):
        return len(self.getArestasSaida())+ len(self.getArestasEntrada())
    
    def getAdjacentes(self, v):
        listaVerticesAdjacentes = []
        for arestas_de_saida in v.getArestasSaida():
            listaVerticesAdjacentes.append(arestas_de_saida.getvDestino())
        return listaVerticesAdjacentes

    def getMenorValor(self):
        menor = self.getValor()[0]
        for a in self.getValor():
            if a < menor:
                menor = a


In [94]:
class Aresta:
    def __init__(self, vOrigem, vDestino, peso, direcionada=True):
        self.__vOrigem = vOrigem
        self.__vDestino = vDestino
        self.__peso = peso
        self.__direcionada = direcionada
        self.__vOrigem.adicionarAresta(self)
        self.__vDestino.adicionarAresta(self)
        
    def getvOrigem(self):
        return self.__vOrigem
    def getvDestino(self):
        return self.__vDestino
    def getValor(self):
        return self.__peso

In [95]:
from collections import deque
class Grafo:
    def __init__(self, direcionado=True):
        self.__vertices = set()
        self.__arestas  = set()
        self.__direcionado = direcionado
        
    def setVertices(self, vertices):
        self.__vertices = vertices
        
    def setArestas(self, arestas):
        self.__arestas = arestas
        
    def getVertices(self):
        return self.__vertices
    
    def getVerticeByValor(self, valor):
        for v in self.__vertices:
            if v.getValor() == valor:
                return v
        return None
    
    def getArestas(self):
        return self.__arestas
    
    def checkHandShakingLemma(self):
        somaGraus = 0
        for v in self.getVertices():
            somaGraus+= v.getGrau()
        if somaGraus == len(self.getArestas())*2:
            return True
        else:
            return False
        
    def dfs(self, graph, v, visitados=[]):
        if v not in visitados: # se v nao foi visitado
            visitados.append(v) # marca vertice como visitado
        if len(v.getAdjacentes(v)) == 0: # vertice escolhido nao tem adjacentes
            self.dfs(graph, next(iter(graph.getVertices())), visitados) # chamada recursiva pegando o proximo vertice do set
        else: # vertice escolhido tem adjacentes
            for adjacente in v.getAdjacentes(v): #percorre todos os adjacentes a ele
                if adjacente not in visitados: # se um dos adjacentes nao foi visitado
                    self.dfs(graph, adjacente, visitados) # chamada recursiva para cada adjacente
        return visitados
    
    def bfs(self, v, visitados = [], fila = deque([])):
        fila.append(v)  # adiciona o vertice v a fila
        if v not in visitados:  # se vertice v nao esta em visitados
            visitados.append(v)  # adiciona vertice v a visitados
        while fila:  # enquanto houver vertices na fila
            vertice = fila.popleft()  # tira vertice ja visitado da fila
            if len(vertice.getArestasSaida()) == 0: # vertice escolhido nao tem adjacentes
                self.bfs(next(iter(self.getVertices())), visitados, fila) # chamada recursiva pegando o proximo vertice do set   
            else:
                for adjacente in vertice.getAdjacentes(v): #percorre todos os adjacentes a ele
                    if adjacente not in visitados: # se um dos adjacentes nao foi visitado
                        visitados.append(adjacente)  # insere o adjacente em visitados
                        self.bfs(adjacente, visitados, fila) # chamada recursiva pegando o proximo vertice do set
        return visitados  # retorna a lista de visitados  
    
    def buscarPorValor(self, valor):
        for v in self.bfs(next(iter(self.getVertices())), visitados = [], fila = deque([])):
            if valor == v.getValor():
                return valor
        return None
    
    def eulerPath(self, lista = [], par = [], impar = []):
        for v in self.getVertices():
            lista.append(v.getGrau())
            
        for l in lista:
            if l % 2 == 0:
                par.append(l)
            elif l % 2 == 1:
                impar.append(l)
                
        if len(impar) == 2 or len(impar) == 0:
            return "é um caminho Euler."
        else:
            return "não é um caminho Euler."
        
    def inserirVertices(self, v):
        if self.buscarPorValor(v) != v:
            self.__vertices.add(Vertice(v))
            return True
        else:
            return False
            
            
    def inserirAresta(self, vO, vD, peso = 1, direcionado = True):
        verticeO = self.getVerticeByValor(vO)
        verticeD = self.getVerticeByValor(vD)
        if verticeO and verticeD is None:
            raise Exception("Esse Vertice nao existe")
            return False
        else:
            self.__arestas.add(Aresta(verticeO, verticeD, peso, direcionado))
            return True
        
        
    def removerAresta(self, vO, vD, peso, direcionado = True):
        verticeO = self.getVerticeByValor(vO)
        verticeD = self.getVerticeByValor(vD)
        if verticeO and verticeD != None:
            for aresta in self.getArestas():
                if verticeO == aresta.getvOrigem() and verticeD == aresta.getvDestino() and peso == aresta.getValor():
                    self.__arestas.remove(aresta)
                    return True
                    
        return False
    
    def removerVertice(self, valor):        
        para_remover = self.getVerticeByValor(valor)
        if para_remover == None:
            pass
        else:
            self.getVertices().remove(para_remover)
            
            for aresta in self.getArestas():
                peso = aresta.getValor()
                vO = aresta.getvOrigem()
                vD = aresta.getvDestino()
                
                if vO == para_remover or vD == para_remover:
                    break
            self.removerAresta(vO, vD, peso)
            
    def matriz(self):
        
        tamanho = len(self.getVertices())
        mAux = [0] * tamanho
        m = []
        for c in range(0, len(mAux)):
            m.append(mAux.copy())
            
        c = 0
        for aresta in self.getArestas():
            m[c][c] = aresta.getvOrigem().getValor()
            m[c] = aresta.getvDestino().getValor()
            c+=1
            
        return m
                

    def dijkstra(self, origem):
        vMenorValor = Vertice.getMenorValor()
        peso = {}
        antecessor = {}
        V = []        
        for v_i in self.getVertices():
            V.append(v_i) 
        for v in V:
            peso[v] = 9999999999999 
            antecessor[v] = 9999999999999 
        peso[origem] = 0 
        while V:        
            V.remove(vMenorValor) 
            for e in self.getVertice(vMenorValor).getArestasSaida(): 
                pesoSomadoParaComparacao = peso[vMenorValor] + e.getPeso() 
                if pesoSomadoParaComparacao < peso[e.getVerticeDestino().getValor()]:
                    peso[e.getVerticeDestino().getValor()] = pesoSomadoParaComparacao 
                    antecessor[e.getVerticeDestino().getValor()] = vMenorValor
        return peso, antecessor
                
                
    def prim(self):
        vertices = []
        
        for v in self.getVertices():
            vertices.append(v)
            
        arvore = []
        
        for valor in vertices:
            menor = 0
            for aresta in self.getVertice(valor).getArestasSaida():
                if aresta:
                    if aresta.getValor() < menor:
                        menor = aresta.getValor()
            for aresta in self.getVertice(valor).getArestasSaida():
                if aresta and aresta.getValor() == menor:
                    arvore.append( [valor, aresta.getValor(), aresta.getvDestino().getValor()] )
        return arvore
    
    def kruskal(self):
        arestas = []
        arvore = []
        
        for v in self.getVertices():
            
            for a in self.getArestas():
                peso_minimo = 0
                if a.getValor() < peso_minimo:
                    peso_minimo = a.getValor()
                    
                if a.getValor() > peso_minimo:
                    arestas.append(a)
                    
            for aresta in arestas:
                if aresta.getvDestino() and aresta.getvOrigem:
                    arvore.append(aresta)
                    
        return arvore

In [96]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True )

In [97]:
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5})
G.setArestas({a1, a2, a3, a4, a5})

In [98]:
for v in G.getVertices():
    print(v.getValor(), end="\t")

5	4	1	2	3	

In [99]:
for a in G.getArestas():
    print(a.getvOrigem().getValor(), end="")
    print(" ---> ", end="")
    print(a.getvDestino().getValor(), end="\t")

2 ---> 3	4 ---> 5	1 ---> 2	3 ---> 4	4 ---> 1	

In [100]:
v1.getGrau()

2

In [101]:
G.checkHandShakingLemma()

True

#### Testando busca em profundidade

In [102]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
v6 = Vertice(6)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True ) 
a6 = Aresta( v4, v6, 60, True ) 
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5,v6})
G.setArestas({a1, a2, a3, a4, a5, a6})
for vertice in G.getVertices():
    print(f"Busca em profundidade, iniciando com o vértice {vertice.getValor()}:")
    for v in G.dfs(G, vertice, visitados=[]):
        print(str(v.getValor())+"\t", end="")
    print("\n.................................................")  

Busca em profundidade, iniciando com o vértice 1:
1	2	3	4	6	5	
.................................................
Busca em profundidade, iniciando com o vértice 5:
5	1	2	3	4	6	
.................................................
Busca em profundidade, iniciando com o vértice 6:
6	1	2	3	4	5	
.................................................
Busca em profundidade, iniciando com o vértice 4:
4	1	2	3	6	5	
.................................................
Busca em profundidade, iniciando com o vértice 3:
3	4	1	2	6	5	
.................................................
Busca em profundidade, iniciando com o vértice 2:
2	3	4	1	6	5	
.................................................


In [103]:
#valor = int(input("Valor procurado: "))
#if G.buscarPorValor(valor) == None:
#    print(f"{valor} não encontrado no grafo")
#else:
#    print(f"{valor} encontrado no grafo")
[print(f"{v.getValor()} encontrado no grafo.") for v in  G.bfs(v1, visitados = [], fila = deque([])) if v == v.getValor() ]

[]

In [104]:
print(G.eulerPath())

é um caminho Euler.


In [105]:
print(G.inserirVertices(20))

True


In [106]:
print(G.inserirVertices(90))

True


In [107]:
G.inserirAresta(20, 90)

True

In [108]:
#G.removerAresta(1, 2, 1)

In [109]:
G.removerVertice(90)

In [110]:
for v in G.getVertices():
    print(v.getValor())

1
5
6
4
3
2
20


In [111]:
for a in G.getArestas():
    print(a.getValor())

30
50
40
60
1
20
10


In [112]:
G.matriz()

[4, 5, 1, 6, 90, 3, 2]

In [113]:
print(G.dijkstra(3))

TypeError: getMenorValor() missing 1 required positional argument: 'self'